# The Battle of Neighbourhoods
## Table of content
### <a>Introduction</a>
### <a>Data</a>
### <a>Methodology</a>
### <a>Results</a>
### <a>Discussion</a>
### <a>Conclusion</a>

# <a>Introduction</a>
## Description & Discussion
The problem here is not a business but a tea time for relaxation. When the time has come to get some refresh, we need to make something changes. Yangon (formerly known as Rangoon) is the largest city in Myanmar (formerly Burma). The former capital city of Myanmar – Yangon – is definitely among the most beautiful cities in Myanmar. As the city goes growing well with population, building and ecosystem, there are more companies to get involved into business. But the most interested thing among young people is to get refresh after having hard work. So the best recommend here is tea shop for youth who loves tea or caffee.

## Description of the data & Tips to solve
Firstly, Exploring location of the city is really important in which there are many townships in the city. So, having a data set something like postal codes is really helpful to visualize and analyze.
### Things to get started: 
#### 1. <a>Setting up for environments</a>
#### 2. <a>Web Scraping</a>
#### 3. <a>Data Cleaning</a> 
#### 4. <a>Explore a location of target area</a>
#### 5. <a>Tea Time</a>

# <a>Data</a>
## 1.Setting up for environments

In [2]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter
from IPython.display import Image 
from IPython.core.display import HTML 
from sklearn.cluster import KMeans
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## 2.Web Scraping

In [3]:
url = 'https://www.myanmarpages.com/emergency-contacts/yangon-region-postal-codes/'

req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11'})

webpage = urlopen(req).read()

content = BeautifulSoup(webpage, 'lxml')

data = []
columns = []

for index, tr in enumerate(content.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#Adding into pandas dataframe
yangon = pd.DataFrame(data = data,columns = columns)
yangon

,,,
0,Names,Township,Postal Codes
1,Ahlone,Ahlone,11121
2,Aung San,Insein,11012
3,Bahan,Bahan,11201
4,Bayintnaung Market (Thamine),Mayangone,11062
5,Bogyoke Market,Pabedan,11143
6,Botahtaung,Botahtaung,11162
7,Botahtaung,Botahtaung,11161
8,Dagon,Dagon,11191
9,Dagon Myothit (North),Dagon Myothit (North),11421


## 3.Data Cleaning

In [4]:
#Saving DataFrame as CSV file, then droping unwanted column and renaming columns to custom 
yangon.to_csv('week4.csv')
ygn = pd.read_csv('week4.csv')
ygn.drop(['Unnamed: 0'], axis = 1, inplace = True)
ygn.drop([0], axis = 0, inplace = True)
ygn.rename(columns= {'Unnamed: 1' : 'Names', 'Unnamed: 2': 'Township', "Unnamed: 3": 'Postal Codes'}, inplace = True)
ygn.head()

,Names,Township,Postal Codes
1,Ahlone,Ahlone,11121
2,Aung San,Insein,11012
3,Bahan,Bahan,11201
4,Bayintnaung Market (Thamine),Mayangone,11062
5,Bogyoke Market,Pabedan,11143


# <a>Methodology</a>
Now, we have data set of Yangon region by scraping the web. In order to establish the targeted township, we will explore the demographics of the township in the city of Yangon by segmenting the data and conducting descriptive analysis using panda. And then, we will customize ,especially venues of tea shops in the targeted township, what we want to explore with foursquare API. The data set will be filtered using Pandas.

## 4.Explore a location of target area
### Foursquare API
### Setting up for credentails: Client ID & Secret 

In [5]:
# @ hidden_cell
CLIENT_ID = 'QZJTDZHQWQSN2L5CKN5L5AXCJRXTRFOCMRHVNNHK445NWBDU' #  Foursquare ID
CLIENT_SECRET = 'NKLGB54WTNFRZZTUXZPXR2AQUC13X2ROTOMBB0UTTFUKKV03' #  Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QZJTDZHQWQSN2L5CKN5L5AXCJRXTRFOCMRHVNNHK445NWBDU
CLIENT_SECRET:NKLGB54WTNFRZZTUXZPXR2AQUC13X2ROTOMBB0UTTFUKKV03


### Geocoding addresses from Pandas DataFrame

In [6]:
geolocator = Nominatim(user_agent="foursquare_agent")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
ygn['location'] = ygn['Names'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
ygn['point'] = ygn['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
ygn[['latitude', 'longitude', 'altitude']] = pd.DataFrame(ygn['point'].tolist(), index=ygn.index)

ygn.head()

,Names,Township,Postal Codes,location,point,latitude,longitude,altitude
1,Ahlone,Ahlone,11121,"(အလုံ, Western District, ရန်ကုန်တိုင်းဒေသကြီး,...","(16.786798, 96.1287706, 0.0)",16.786798,96.128771,0.0
2,Aung San,Insein,11012,"(Aung, Kecamatan Galang, Tolitoli, Sulawesi Te...","(1.1806, 120.8271, 0.0)",1.180600,120.827100,0.0
3,Bahan,Bahan,11201,"(Bahan, Karimun, Kepulauan Riau, Indonesia, (0...","(0.893889, 103.794444, 0.0)",0.893889,103.794444,0.0
4,Bayintnaung Market (Thamine),Mayangone,11062,None,None,NaN,NaN,NaN
5,Bogyoke Market,Pabedan,11143,"(ဗိုလ်ချုပ်ဈေး, ပေါင်, Thaton District, Mon, M...","(16.6912458, 97.4263586, 0.0)",16.691246,97.426359,0.0


In [7]:
#Data Cleaning
ygn_geo = ygn.copy()
ygn_geo.drop(['location', 'point', 'altitude'], axis = "columns", inplace = True)
ygn_geo.dropna(subset= ["latitude","longitude"] , inplace = True)
ygn_geo.head()

,Names,Township,Postal Codes,latitude,longitude
1,Ahlone,Ahlone,11121,16.786798,96.128771
2,Aung San,Insein,11012,1.180600,120.827100
3,Bahan,Bahan,11201,0.893889,103.794444
5,Bogyoke Market,Pabedan,11143,16.691246,97.426359
6,Botahtaung,Botahtaung,11162,16.768490,96.171971


In [8]:
ygn_geo

,Names,Township,Postal Codes,latitude,longitude
1,Ahlone,Ahlone,11121,16.786798,96.128771
2,Aung San,Insein,11012,1.180600,120.827100
3,Bahan,Bahan,11201,0.893889,103.794444
5,Bogyoke Market,Pabedan,11143,16.691246,97.426359
6,Botahtaung,Botahtaung,11162,16.768490,96.171971
7,Botahtaung,Botahtaung,11161,16.768490,96.171971
8,Dagon,Dagon,11191,38.344636,-120.961330
9,Dagon Myothit (North),Dagon Myothit (North),11421,16.878127,96.204913
10,Dagon Myothit (South),Dagon Myothit (South),11431,16.854924,96.223217
11,Dawbon,Dawbon,11241,16.782625,96.187776


## 5.Tea Time

In [9]:
#Define a query to search for tea
search_query = 'tea'
radius = 1000  
print(search_query + ' .... OK!')

tea .... OK!


In [10]:
# Target area-  Row No.25: Pabedan
latitude = '16.775958' #latitude of Pabedan
longitude = '96.156149' #longitude of Pabedan

In [11]:
# Define the corresponding URL

url = 'https://api.foursquare.com/v2/venues/search?client_id=QZJTDZHQWQSN2L5CKN5L5AXCJRXTRFOCMRHVNNHK445NWBDU&client_secret=NKLGB54WTNFRZZTUXZPXR2AQUC13X2ROTOMBB0UTTFUKKV03&ll=16.775958,96.156149&v=20180604&query=tea&radius=5000&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QZJTDZHQWQSN2L5CKN5L5AXCJRXTRFOCMRHVNNHK445NWBDU&client_secret=NKLGB54WTNFRZZTUXZPXR2AQUC13X2ROTOMBB0UTTFUKKV03&ll=16.775958,96.156149&v=20180604&query=tea&radius=5000&limit=30'

In [12]:
#Send the GET Request and examine the results
results = requests.get(url).json()

In [13]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood
0,5a446b4309e283441fbafb8a,Akkhaya Tea House,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1598621632,False,Manaw Hari Road,Pyay Road,16.792136,96.140456,"[{'label': 'display', 'lat': 16.792136, 'lng':...",2457,MM,Yangon,Yangon Region,မြန်မာ,"[Manaw Hari Road (Pyay Road), Yangon, Yangon R...",NaN,NaN
1,55536c06498e7d34ddd21e28,Tea King Tea Production,"[{'id': '4bf58dd8d48988d1f7941735', 'name': 'F...",v-1598621632,False,"No (190), Konzaydan Street",Mahar bandula Street,16.774773,96.154640,"[{'label': 'display', 'lat': 16.77477264404297...",208,MM,Yangon,Yangon Region,မြန်မာ,"[No (190), Konzaydan Street (Mahar bandula Str...",NaN,NaN
2,5e40fc9474e3de000764d6e6,Tea Talk,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",v-1598621632,False,"Room No. (9), Naing Group Tower (1), Anawraht...",Between Sule Pagoda Road & 32th Street,16.776705,96.158400,"[{'label': 'display', 'lat': 16.776705, 'lng':...",253,MM,Yangon,Yangon Region,မြန်မာ,"[Room No. (9), Naing Group Tower (1), Anawrah...",NaN,NaN
3,513930afe4b084437e6b863c,Golden Tea,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1598621632,False,NaN,NaN,16.773934,96.157123,"[{'label': 'display', 'lat': 16.77393441547805...",248,MM,NaN,NaN,မြန်မာ,[မြန်မာ],NaN,NaN
4,5ae420854b78c5002c1fcd9f,Golden Bell Tea & Food Centre,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1598621632,False,"No. (365), Bogyoke Aung San Street",Pabedan Township,16.779166,96.157645,"[{'label': 'display', 'lat': 16.779166, 'lng':...",391,MM,Yangon,Yangon Region,မြန်မာ,"[No. (365), Bogyoke Aung San Street (Pabedan T...",NaN,NaN


In [14]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,Akkhaya Tea House,Breakfast Spot,Manaw Hari Road,Pyay Road,16.792136,96.140456,"[{'label': 'display', 'lat': 16.792136, 'lng':...",2457,MM,Yangon,Yangon Region,မြန်မာ,"[Manaw Hari Road (Pyay Road), Yangon, Yangon R...",NaN,NaN,5a446b4309e283441fbafb8a
1,Tea King Tea Production,Flea Market,"No (190), Konzaydan Street",Mahar bandula Street,16.774773,96.154640,"[{'label': 'display', 'lat': 16.77477264404297...",208,MM,Yangon,Yangon Region,မြန်မာ,"[No (190), Konzaydan Street (Mahar bandula Str...",NaN,NaN,55536c06498e7d34ddd21e28
2,Tea Talk,Bubble Tea Shop,"Room No. (9), Naing Group Tower (1), Anawraht...",Between Sule Pagoda Road & 32th Street,16.776705,96.158400,"[{'label': 'display', 'lat': 16.776705, 'lng':...",253,MM,Yangon,Yangon Region,မြန်မာ,"[Room No. (9), Naing Group Tower (1), Anawrah...",NaN,NaN,5e40fc9474e3de000764d6e6
3,Golden Tea,Café,NaN,NaN,16.773934,96.157123,"[{'label': 'display', 'lat': 16.77393441547805...",248,MM,NaN,NaN,မြန်မာ,[မြန်မာ],NaN,NaN,513930afe4b084437e6b863c
4,Golden Bell Tea & Food Centre,Noodle House,"No. (365), Bogyoke Aung San Street",Pabedan Township,16.779166,96.157645,"[{'label': 'display', 'lat': 16.779166, 'lng':...",391,MM,Yangon,Yangon Region,မြန်မာ,"[No. (365), Bogyoke Aung San Street (Pabedan T...",NaN,NaN,5ae420854b78c5002c1fcd9f


# <a>Results</a>
With the filtered data set, we can explore a venue map with location of targeted area - Pabedan township. The name of the data frame will be used for exploring the map. This will give the exact label in the map. From this final map, we run k-means to cluster the venues into five clusters.

In [15]:
dataframe_filtered.name

0                              Akkhaya Tea House
1                        Tea King Tea Production
2                                       Tea Talk
3                                     Golden Tea
4                  Golden Bell Tea & Food Centre
5                     Shwe Khaung Laung Tea Shop
6                               Acacia Tea Salon
7                                           iTea
8              35th St Tea Shop ၃၅ လမ်းလ္ဘက်ရည်။
9                                      Tea Black
10                             Rangoon Tea House
11                                     Tea Black
12                        Shwe Wel Tun Tea House
13                              Kumudra Tea Shop
14                       Shwe Pann Thee tea shop
15                                 Boss Tea Shop
16                         Lucky Flower Tea Shop
17                                 itea tea room
18                            U Kar Ka Green Tea
19    Kyaw Thaung Tea Shop (Bo Yar Nyunt Street)
20                  

### The final map:

In [16]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB dark_matter') # generate map 

tooltip = 'Click me!'
# add a red circle marker to represent
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Pabedan, Center point',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.6,
).add_to(venues_map)

# add the Tea Shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        tooltip=tooltip,
        fill = True,
        fill_color = 'blue',
        fill_opacity=0.6
    ).add_to(venues_map)
# display map
venues_map

## I added a screenshot of this map in the folder

### Clustering tea shops
### The clusters are visualized via a dark matter folium map:

In [17]:
Pabedan_tea_map_area = folium.Map(location=[16.775958,96.156149], zoom_start=12,tiles='CartoDB dark_matter')

X = dataframe_filtered['lat']
Y = dataframe_filtered['lng']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=5, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow', 'pink']
dataframe_filtered['Cluster'] = clusters

for latitude, longitude, address, cluster in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['Cluster']):
    label = folium.Popup(address, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color=colors,
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(Pabedan_tea_map_area)  

Pabedan_tea_map_area

## I added a screenshot of this map in the folder

 # <a>Discussion</a>
 From the results discovered and presented, based on the criteria given by the cluster data, the rating of the tea shop can be explored by far for recommendation to get refresh in weekend or after having some hard work. The best rating score of the tea shop will give the best services as we have known in this century. Additionally, from the clustering, a business can be started for opening another tea shop in this targeted township. In this case, opening another tea shop around competitive ones, there is more cases to lose profits and getting less customer. So this can go worse if the owner does not have competitve mind set. The other things we can explore for getting better business around the targeted area are venues of companies, universities, colleges and so on.

# <a>Conclusion</a>
In conclusion, the scope of the analysis is somewhat limited. The society is ever changing. the information afforded us may be dated due to relying on information via Foursquare. Overall though, the model created can easily be replicated again and again with monitored data via the Foursquare API.